In [1]:
import os
from typing import TypedDict, Annotated, List, Dict, Any

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

from langgraph.graph import StateGraph, END

import operator
import json

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# Define the state variables
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    current_agent: str
    research_data: dict
    analysis_complete: bool
    final_report: str

In [5]:
# INitialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=2000
    )

In [6]:
def simulate_web_search(query: str) -> str:
    """Simulated web search - replace with real API in production"""
    return f"Search results for '{query}': Found relevant information about {query} including recent developments, expert opinions, and statistical data."

In [7]:
def simulate_data_analysis(data: str) -> str:
    """Simulated data analysis tool"""
    return f"Analysis complete: Key insights from the data include emerging trends, statistical patterns, and actionable recommendations."

In [8]:
def research_agent(state: AgentState) -> AgentState:
    """Agent that conducts research on a given topic"""
    messages = state["messages"]
    last_message = messages[-1].content

    search_results = simulate_web_search(last_message)

    prompt = f"""You are a research agent. Based on the query: "{last_message}"

    Here are the search results: {search_results}

    Conduct thorough research and gather relevant information. Provide structured findings with:
    1. Key facts and data points
    2. Current trends and developments
    3. Expert opinions and insights
    4. Relevant statistics

    Be comprehensive and analytical in your research summary."""

    response = llm.invoke([HumanMessage(content=prompt)])

    research_data = {
        "topic": last_message,
        "findings": response.content,
        "search_results": search_results,
        "sources": ["academic_papers", "industry_reports", "expert_analyses"],
        "confidence": 0.88,
        "timestamp": "2024-research-session"
    }

    return {
        "messages": state["messages"] + [AIMessage(content=f"Research completed on '{last_message}': {response.content}")],
        "current_agent": "analysis",
        "research_data": research_data,
        "analysis_complete": False,
        "final_report": ""
    }

In [9]:

def analysis_agent(state: AgentState) -> AgentState:
    """Agent that analyzes research data and extracts insights"""
    research_data = state["research_data"]

    analysis_results = simulate_data_analysis(research_data.get('findings', ''))

    prompt = f"""You are an analysis agent. Analyze this research data in depth:

    Topic: {research_data.get('topic', 'Unknown')}
    Research Findings: {research_data.get('findings', 'No findings')}
    Analysis Results: {analysis_results}

    Provide deep insights including:
    1. Pattern identification and trend analysis
    2. Comparative analysis with industry standards
    3. Risk assessment and opportunities
    4. Strategic implications
    5. Actionable recommendations with priority levels

    Be analytical and provide evidence-based insights."""

    response = llm.invoke([HumanMessage(content=prompt)])

    return {
        "messages": state["messages"] + [AIMessage(content=f"Analysis completed: {response.content}")],
        "current_agent": "report",
        "research_data": state["research_data"],
        "analysis_complete": True,
        "final_report": ""
    }

In [10]:
def report_agent(state: AgentState) -> AgentState:
    """Agent that generates final comprehensive reports"""
    research_data = state["research_data"]

    analysis_message = None
    for msg in reversed(state["messages"]):
        if isinstance(msg, AIMessage) and "Analysis completed:" in msg.content:
            analysis_message = msg.content.replace("Analysis completed: ", "")
            break

    prompt = f"""You are a professional report generation agent. Create a comprehensive executive report based on:

    🔍 Research Topic: {research_data.get('topic')}
    📊 Research Findings: {research_data.get('findings')}
    🧠 Analysis Results: {analysis_message or 'Analysis pending'}

    Generate a well-structured, professional report with these sections:

    ## EXECUTIVE SUMMARY
    [Provide a concise overview of key findings and recommendations]

    ## KEY RESEARCH FINDINGS
    [Detail the most important discoveries and data points]

    ## ANALYTICAL INSIGHTS
    [Present deep analysis, patterns, and trends identified]

    ## STRATEGIC RECOMMENDATIONS
    [Provide actionable recommendations with priority levels]

    ## RISK ASSESSMENT & OPPORTUNITIES
    [Identify potential risks and opportunities]

    ## CONCLUSION & NEXT STEPS
    [Summarize and suggest follow-up actions]

    Make the report professional, data-driven, and actionable."""

    response = llm.invoke([HumanMessage(content=prompt)])

    return {
        "messages": state["messages"] + [AIMessage(content=f"📄 FINAL REPORT GENERATED:\n\n{response.content}")],
        "current_agent": "complete",
        "research_data": state["research_data"],
        "analysis_complete": True,
        "final_report": response.content
    }

In [11]:
def should_continue(state: AgentState) -> str:
    """Determine which agent should run next based on current state"""
    current_agent = state.get("current_agent", "research")

    if current_agent == "research":
        return "analysis"
    elif current_agent == "analysis":
        return "report"
    elif current_agent == "report":
        return END
    else:
        return END

In [12]:
workflow = StateGraph(AgentState)

workflow.add_node("research", research_agent)
workflow.add_node("analysis", analysis_agent)
workflow.add_node("report", report_agent)

workflow.add_conditional_edges(
    "research",
    should_continue,
    {"analysis": "analysis", END: END}
)

workflow.add_conditional_edges(
    "analysis",
    should_continue,
    {"report": "report", END: END}
)

workflow.add_conditional_edges(
    "report",
    should_continue,
    {END: END}
)

workflow.set_entry_point("research")

app = workflow.compile()

In [13]:
def run_research_assistant(query: str):
    """Run the complete research workflow"""
    initial_state = {
        "messages": [HumanMessage(content=query)],
        "current_agent": "research",
        "research_data": {},
        "analysis_complete": False,
        "final_report": ""
    }

    print(f"🔍 Starting Multi-Agent Research on: '{query}'")
    print("=" * 60)

    current_state = initial_state

    print("🤖 Research Agent: Gathering information...")
    current_state = research_agent(current_state)
    print("✅ Research phase completed!\n")

    print("🧠 Analysis Agent: Analyzing findings...")
    current_state = analysis_agent(current_state)
    print("✅ Analysis phase completed!\n")

    print("📊 Report Agent: Generating comprehensive report...")
    final_state = report_agent(current_state)
    print("✅ Report generation completed!\n")

    print("=" * 60)
    print("🎯 MULTI-AGENT WORKFLOW COMPLETED SUCCESSFULLY!")
    print("=" * 60)

    final_report = final_state['final_report']
    print(f"\n📋 COMPREHENSIVE RESEARCH REPORT:\n")
    print(final_report)

    return final_state

In [14]:
result = run_research_assistant("What are the latest trends in artificial intelligence for 2025?")

🔍 Starting Multi-Agent Research on: 'What are the latest trends in artificial intelligence for 2025?'
🤖 Research Agent: Gathering information...
✅ Research phase completed!

🧠 Analysis Agent: Analyzing findings...
✅ Analysis phase completed!

📊 Report Agent: Generating comprehensive report...
✅ Report generation completed!

🎯 MULTI-AGENT WORKFLOW COMPLETED SUCCESSFULLY!

📋 COMPREHENSIVE RESEARCH REPORT:

# EXECUTIVE REPORT ON THE LATEST TRENDS IN ARTIFICIAL INTELLIGENCE FOR 2025

## EXECUTIVE SUMMARY
The landscape of artificial intelligence (AI) is poised for unprecedented growth and transformation by 2025. The global AI market is projected to reach approximately $1.5 trillion, driven by a compound annual growth rate (CAGR) of around 20%. Notably, venture capital investment in AI startups is expected to exceed $150 billion, reflecting strong confidence in the technology's future. By 2025, an estimated 80% of businesses will adopt AI technologies across various operations. Key trends in